In [1]:
import pandas as pd

data_source = "/home/fra/DataMart/datacentre/olddata/"
train_file = "adult_train.csv"
test_file = "adult_test.csv"

# load data
df = pd.read_csv(data_source + train_file)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
import sys
mylib = '/home/fra/Project/pyProj/mlib'

In [3]:
sys.path.append(mylib)

In [4]:
exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY']
target_col = 'INCOME'

In [5]:
import utility as util
import explore as ex
import pipeline as pl
import transform as trf
from imp import reload

In [6]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [7]:
stp = pl.StandardPipeline(target_col, exclusions)

In [8]:
stp.process(df, ntransformer=StandardScaler, ctransformer=OrdinalEncoder)

In [9]:
(X, y) = stp.out()

In [234]:
stp.target_col

'INCOME'

In [235]:
stp.mapping

{' <=50K': 0, ' >50K': 1}

### basic model

In [10]:
from sklearn.model_selection import train_test_split
features = df.columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
import xgboost as xgb
model = xgb.XGBClassifier().fit(X_train, y_train)

In [12]:
def print_model_performance(model, X_test, y_test):
    predictions = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)

    print("Accuracy: %f" % accuracy_score(y_test, predictions))
    print("Confustion Matrix: ")
    print(confusion_matrix(y_true=y_test, y_pred=predictions))
    print("AUC Score: %f" % roc_auc_score(y_test, pred_prob[:,1]))
    print("GINI Coefficient: %f" % (roc_auc_score(y_test, pred_prob[:,1])*2-1))

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

In [14]:
print_model_performance(model, X_test, y_test)

Accuracy: 0.870918
Confustion Matrix: 
[[6926  476]
 [ 785 1582]]
AUC Score: 0.926003
GINI Coefficient: 0.852005


### Evaluator

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=0)

In [16]:
import lightgbm
lgbm = lightgbm.LGBMClassifier()

In [17]:
from sklearn.model_selection import cross_val_score, GridSearchCV

### LGBM

In [18]:
param_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 100],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
}

In [19]:
# gsearch = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=3)

In [20]:
gsearch2 = GridSearchCV(estimator=lgbm, param_grid=param_grid, scoring="roc_auc", cv=3)

In [21]:
gs = gsearch2.fit(X, y)

In [22]:
print_model_performance(gs, X_test, y_test)

Accuracy: 0.888423
Confustion Matrix: 
[[7033  369]
 [ 721 1646]]
AUC Score: 0.942842
GINI Coefficient: 0.885683


In [23]:
gs.best_params_

{'lambda_l1': 1,
 'lambda_l2': 0,
 'min_data_in_leaf': 30,
 'num_leaves': 31,
 'reg_alpha': 0.1}

In [24]:
gs.best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', lambda_l1=1, lambda_l2=0,
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_data_in_leaf=30, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.1, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### My own classifier

In [25]:
import model as ml

In [171]:
lgt_grid_param = {'penalty': ['l1', 'l2'], 'C': [0.5, 0.25, 0.1], 'solver': ['liblinear']}
rfm_grid_param = {'n_estimators': [50, 100, 200], 'max_depth': [4, 5], 'max_features': ['auto'], 'criterion': ['gini'], 'min_samples_leaf': [20],  'min_samples_split': [50, 100]}
dtc_grid_param = {'criterion': ['gini'], 'max_depth': [4, 5], 'min_samples_split': [100, 50], 'min_impurity_decrease': [0.0005, 0.00025]}
xgb_grid_param = {'n_estimators': [50, 100], 'max_depth': [3, 4]}
lgbm_grid_param = {
        'n_estimators': [100, 200],
        'max_depth': [5,8],
        'min_child_samples': [20],
        'num_leaves': [32],
        'reg_alpha': [0],
        'learning_rate': [0.01, 0.005, 0.05, 0.1]
    }

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 

### SVC

In [217]:
from sklearn.svm import SVC

In [230]:
svm_grid_param = {'kernel':['linear', 'rbf', 'sigmoid']}

In [222]:
svm = SVC(probability=True)

#### slow....

In [223]:
svm.fit(X_train, y_train)
print_model_performance(svm, X_test, y_test)

Accuracy: 0.806019
Confustion Matrix: 
[[7303   99]
 [1796  571]]
AUC Score: 0.858227
GINI Coefficient: 0.716454


In [227]:
sv = ml.BinaryClassifier(SVC(probability=True))

In [231]:
sv.tune(X_train, y_train, svm_grid_param, cv=3)

Best params:
{'kernel': 'linear'}


BinaryClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                               class_weight=None, coef0=0.0,
                               decision_function_shape='ovr', degree=3,
                               gamma='scale', kernel='linear', max_iter=-1,
                               probability=True, random_state=None,
                               shrinking=True, tol=0.001, verbose=False))

In [232]:
print_model_performance(sv, X_test, y_test)

Accuracy: 0.814311
Confustion Matrix: 
[[7220  182]
 [1632  735]]
AUC Score: 0.850056
GINI Coefficient: 0.700111


In [233]:
sv.estimator.coef_

array([[ 1.48846292e-01, -4.10884550e-03,  1.80259571e-02,
         5.71144739e-03,  4.01793170e-01, -8.37197868e-02,
         1.20062370e-03, -7.35706177e-02,  1.74471782e-02,
         3.28341264e-01,  1.80738359e+00,  2.39481919e-01,
         1.37713586e-01,  1.23731916e-04]])

### LGR

In [206]:
reload(ml)

<module 'model' from '/home/fra/Project/pyProj/mlib/model.py'>

In [200]:
lgr_grid_param = {'penalty': ['l1', 'l2', 'elasticnet'], 'max_iter':[100, 200]}

In [193]:
lgr = LogisticRegression()

In [194]:
lgr.fit(X_train, y_train)
print_model_performance(lgr, X_test, y_test)

Accuracy: 0.823114
Confustion Matrix: 
[[6979  423]
 [1305 1062]]
AUC Score: 0.856598
GINI Coefficient: 0.713196


/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [207]:
lg = ml.BinaryClassifier(LogisticRegression())

In [208]:
lg.tune(X_train, y_train, lgr_grid_param, cv=3)

/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940:

Best params:
{'max_iter': 100, 'penalty': 'l2'}


/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


BinaryClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                              dual=False, fit_intercept=True,
                                              intercept_scaling=1,
                                              l1_ratio=None, max_iter=100,
                                              multi_class='auto', n_jobs=None,
                                              penalty='l2', random_state=None,
                                              solver='lbfgs', tol=0.0001,
                                              verbose=0, warm_start=False))

In [209]:
print_model_performance(lg, X_test, y_test)

Accuracy: 0.823114
Confustion Matrix: 
[[6979  423]
 [1305 1062]]
AUC Score: 0.856598
GINI Coefficient: 0.713196


In [212]:
lg.estimator.coef_

array([[ 4.37233414e-01, -1.63957408e-02,  4.49640021e-02,
         9.16015410e-03,  8.44194918e-01, -2.45366423e-01,
         1.01260613e-02, -1.22996003e-01,  4.56098048e-02,
         7.92701057e-01,  2.56419649e+00,  2.67068335e-01,
         3.58577067e-01, -9.28546716e-04]])

In [210]:
lg.key_drivers()

array([], dtype=float64)

In [224]:
lg.cross_validate(X_train, y_train, cv=3, scoring="roc_auc")

/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org

3-Fold score: 0.852 +/- 0.002


/home/fra/Project/pyProj/pyml/pymlenv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0.85268199, 0.84929997, 0.85302725])

### KNN

In [187]:
knn_grid_param = {'n_neighbors': [5, 10, 20, 50, 100], 'leaf_size': [20, 30, 40]}

In [179]:
knn = KNeighborsClassifier()

In [180]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [181]:
print_model_performance(knn, X_test, y_test)

Accuracy: 0.824445
Confustion Matrix: 
[[6675  727]
 [ 988 1379]]
AUC Score: 0.838926
GINI Coefficient: 0.677852


In [183]:
knnc = ml.BinaryClassifier(KNeighborsClassifier())

In [184]:
knnc.fit(X_train, y_train)

BinaryClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                                metric='minkowski',
                                                metric_params=None, n_jobs=None,
                                                n_neighbors=5, p=2,
                                                weights='uniform'))

In [185]:
print_model_performance(knnc, X_test, y_test)

Accuracy: 0.824445
Confustion Matrix: 
[[6675  727]
 [ 988 1379]]
AUC Score: 0.838926
GINI Coefficient: 0.677852


In [188]:
knnc.tune(X_train, y_train, knn_grid_param, cv=3)

Best params:
{'leaf_size': 20, 'n_neighbors': 20}


BinaryClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=20,
                                                metric='minkowski',
                                                metric_params=None, n_jobs=None,
                                                n_neighbors=20, p=2,
                                                weights='uniform'))

In [189]:
print_model_performance(knnc, X_test, y_test)

Accuracy: 0.833146
Confustion Matrix: 
[[6888  514]
 [1116 1251]]
AUC Score: 0.870748
GINI Coefficient: 0.741495


In [190]:
knnc.cross_validate(X_train, y_train, cv=3, scoring="roc_auc")

3-Fold score: 0.872 +/- 0.003


array([0.87369958, 0.87431483, 0.86741825])

In [191]:
knnc.key_drivers()

AttributeError: 'KNeighborsClassifier' object has no attribute 'feature_importances_'

### XGB

In [85]:
xgbc = xgb.XGBClassifier()

In [86]:
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [87]:
print_model_performance(rfm, X_test, y_test)

Accuracy: 0.859863
Confustion Matrix: 
[[6900  502]
 [ 867 1500]]
AUC Score: 0.907019
GINI Coefficient: 0.814039


In [88]:
xg = ml.BinaryClassifier(xgb.XGBClassifier())

In [89]:
xg.fit(X_train, y_train)

BinaryClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                         colsample_bylevel=1,
                                         colsample_bynode=1, colsample_bytree=1,
                                         gamma=0, gpu_id=-1,
                                         importance_type='gain',
                                         interaction_constraints='',
                                         learning_rate=0.300000012,
                                         max_delta_step=0, max_depth=6,
                                         min_child_weight=1, missing=nan,
                                         monotone_constraints='()',
                                         n_estimators=100, n_jobs=0,
                                         num_parallel_tree=1,
                                         objective='binary:logistic',
                                         random_state=0, reg_alpha=0,
                                  

In [90]:
print_model_performance(xg, X_test, y_test)

Accuracy: 0.870918
Confustion Matrix: 
[[6926  476]
 [ 785 1582]]
AUC Score: 0.926003
GINI Coefficient: 0.852005


In [91]:
xg.tune(X_train, y_train, xgb_grid_param, cv=3)

Best params:
{'max_depth': 4, 'n_estimators': 100}


BinaryClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                         colsample_bylevel=1,
                                         colsample_bynode=1, colsample_bytree=1,
                                         gamma=0, gpu_id=-1,
                                         importance_type='gain',
                                         interaction_constraints='',
                                         learning_rate=0.300000012,
                                         max_delta_step=0, max_depth=4,
                                         min_child_weight=1, missing=nan,
                                         monotone_constraints='()',
                                         n_estimators=100, n_jobs=0,
                                         num_parallel_tree=1,
                                         objective='binary:logistic',
                                         random_state=0, reg_alpha=0,
                                  

In [92]:
print_model_performance(xg, X_test, y_test)

Accuracy: 0.875729
Confustion Matrix: 
[[6980  422]
 [ 792 1575]]
AUC Score: 0.929167
GINI Coefficient: 0.858333


In [93]:
xg.cross_validate(X_train, y_train, cv=3, scoring="roc_auc")

3-Fold score: 0.924 +/- 0.002


array([0.92318445, 0.92291549, 0.9270283 ])

In [94]:
xg.key_drivers()

array([0.05020521, 0.01841816, 0.01155579, 0.00918105, 0.1636247 ,
       0.1074195 , 0.03571   , 0.3522167 , 0.01344375, 0.0344901 ,
       0.11035461, 0.0505986 , 0.03096655, 0.01181528], dtype=float32)

### LGBM

In [145]:
reload(ml)

<module 'model' from '/home/fra/Project/pyProj/mlib/model.py'>

In [146]:
from lightgbm import LGBMClassifier

In [159]:
lgb = LGBMClassifier()

In [160]:
lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [165]:
print_model_performance(lgb, X_test, y_test)

Accuracy: 0.877981
Confustion Matrix: 
[[6996  406]
 [ 786 1581]]
AUC Score: 0.928193
GINI Coefficient: 0.856385


In [172]:
lgbm = ml.BinaryClassifier(LGBMClassifier())

In [173]:
lgbm.fit(X_train, y_train)

BinaryClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                          class_weight=None,
                                          colsample_bytree=1.0,
                                          importance_type='split',
                                          learning_rate=0.1, max_depth=-1,
                                          min_child_samples=20,
                                          min_child_weight=0.001,
                                          min_split_gain=0.0, n_estimators=100,
                                          n_jobs=-1, num_leaves=31,
                                          objective=None, random_state=None,
                                          reg_alpha=0.0, reg_lambda=0.0,
                                          silent=True, subsample=1.0,
                                          subsample_for_bin=200000,
                                          subsample_freq=0))

In [174]:
print_model_performance(lgbm, X_test, y_test)

Accuracy: 0.877981
Confustion Matrix: 
[[6996  406]
 [ 786 1581]]
AUC Score: 0.928193
GINI Coefficient: 0.856385


In [175]:
lgbm.tune(X_train, y_train, lgbm_grid_param, cv=3)

Best params:
{'learning_rate': 0.1, 'max_depth': 5, 'min_child_samples': 20, 'n_estimators': 200, 'num_leaves': 32, 'reg_alpha': 0}


BinaryClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                          class_weight=None,
                                          colsample_bytree=1.0,
                                          importance_type='split',
                                          learning_rate=0.1, max_depth=5,
                                          min_child_samples=20,
                                          min_child_weight=0.001,
                                          min_split_gain=0.0, n_estimators=200,
                                          n_jobs=-1, num_leaves=32,
                                          objective=None, random_state=None,
                                          reg_alpha=0, reg_lambda=0.0,
                                          silent=True, subsample=1.0,
                                          subsample_for_bin=200000,
                                          subsample_freq=0))

In [176]:
print_model_performance(lgbm, X_test, y_test)

Accuracy: 0.875013
Confustion Matrix: 
[[6969  433]
 [ 788 1579]]
AUC Score: 0.929290
GINI Coefficient: 0.858580


In [177]:
lgbm.cross_validate(X_train, y_train, cv=3, scoring="roc_auc")

3-Fold score: 0.926 +/- 0.002


array([0.92407029, 0.92447826, 0.92796131])

In [178]:
lgbm.key_drivers()

array([718, 221, 676, 181, 408, 243, 386, 192,  75,  58, 504, 316, 453,
       126], dtype=int32)

### RFM

In [69]:
rfm = RandomForestClassifier()

In [70]:
rfm.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [71]:
print_model_performance(rfm, X_test, y_test)

Accuracy: 0.859863
Confustion Matrix: 
[[6900  502]
 [ 867 1500]]
AUC Score: 0.907019
GINI Coefficient: 0.814039


In [72]:
rf = ml.BinaryClassifier(RandomForestClassifier())

In [73]:
rf.fit(X_train, y_train)

BinaryClassifier(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  max_samples=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100, n_jobs=None,
                                                  oo

In [74]:
print_model_performance(rf, X_test, y_test)

Accuracy: 0.860068
Confustion Matrix: 
[[6898  504]
 [ 863 1504]]
AUC Score: 0.905734
GINI Coefficient: 0.811469


In [82]:
rf.tune(X_train, y_train, rfm_grid_param, cv=3)

Best params:
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'auto', 'min_samples_leaf': 20, 'min_samples_split': 100, 'n_estimators': 100}


BinaryClassifier(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini', max_depth=5,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  max_samples=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=20,
                                                  min_samples_split=100,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100, n_jobs=None,
                                                  oob_score=False,
                                   

In [83]:
print_model_performance(rf, X_test, y_test)

Accuracy: 0.847477
Confustion Matrix: 
[[7078  324]
 [1166 1201]]
AUC Score: 0.900314
GINI Coefficient: 0.800627


In [84]:
rf.cross_validate(X_train, y_train, cv=3, scoring="roc_auc")

3-Fold score: 0.902 +/- 0.001


array([0.90038137, 0.90287339, 0.90219746])

### DTC

In [28]:
dt = DecisionTreeClassifier()

In [29]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [30]:
print_model_performance(dt, X_test, y_test)

Accuracy: 0.813492
Confustion Matrix: 
[[6441  961]
 [ 861 1506]]
AUC Score: 0.753209
GINI Coefficient: 0.506419


In [58]:
dtc = ml.BinaryClassifier(DecisionTreeClassifier())

In [59]:
dtc.fit(X_train, y_train)

BinaryClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  presort='deprecated',
                                                  random_state=None,
                                                  splitter='best'))

In [60]:
dtc.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [61]:
print_model_performance(dtc, X_test, y_test)

Accuracy: 0.815437
Confustion Matrix: 
[[6447  955]
 [ 848 1519]]
AUC Score: 0.756361
GINI Coefficient: 0.512721


### Tune

In [62]:
dtc.tune(X_train, y_train, dtc_grid_param, cv=10)

Best params:
{'criterion': 'gini', 'max_depth': 5, 'min_impurity_decrease': 0.0005, 'min_samples_split': 100}


BinaryClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini', max_depth=5,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0005,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=100,
                                                  min_weight_fraction_leaf=0.0,
                                                  presort='deprecated',
                                                  random_state=None,
                                                  splitter='best'))

In [63]:
dtc.estimator

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0005, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [64]:
dtc._base_estimator

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0005, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [65]:
print_model_performance(dtc, X_test, y_test)

Accuracy: 0.851367
Confustion Matrix: 
[[7015  387]
 [1065 1302]]
AUC Score: 0.872360
GINI Coefficient: 0.744721


In [67]:
dtc.cross_validate(X_train, y_train, cv=10, scoring="roc_auc")

10-Fold score: 0.876 +/- 0.011


array([0.88249208, 0.87369302, 0.86696016, 0.86997095, 0.88964792,
       0.86544969, 0.89308679, 0.89105492, 0.87147827, 0.85935937])

In [68]:
cross_val_score(estimator=dt, X=X_train, y=y_train, cv=10, scoring = "roc_auc")

array([0.760302  , 0.74080638, 0.73636025, 0.75068839, 0.73731639,
       0.73414193, 0.7477565 , 0.74424533, 0.73611416, 0.73851915])

In [43]:
cross_val_score(estimator=dtc.estimator, X=X_train, y=y_train, cv=3, scoring = "roc_auc")

array([0.87313825, 0.88554315, 0.87354031])

### RFM